In [ ]:
import pandas as pd
import numpy as np

# 1. CSVファイルからデータを読み込む
# 日経平均株価データを 'japan_2022.csv' から読み込む
df = pd.read_csv('japan_2022.csv')

# データの先頭5行を表示して確認する
print("データの先頭5行:")
print(df.head())

In [ ]:
# 欠損値の確認
# 各列の欠損値の数を表示
print("\n欠損値の確認:")
print(df.isnull().sum())


# 2. 欠損値の補完
# 'ffill' は前の行の値で欠損値を埋める方法
df.fillna(method='ffill', inplace=True)

# 欠損値の補完後、再度欠損値の確認
print("\n欠損値補完後の確認:")
print(df.isnull().sum())


# 3. 移動平均の計算
# 5日, 10日, 20日の移動平均を計算して、特徴量として追加
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_20'] = df['Close'].rolling(window=20).mean()


# 4. ボラティリティの計算
# 5日のボラティリティ（標準偏差）を計算して、特徴量として追加
df['Volatility'] = df['Close'].rolling(window=5).std()


# 5. 時間に関連する特徴量の追加
# 'Date'列を日時型に変換
df['Date'] = pd.to_datetime(df['Date'])

# 曜日 (0=月曜日, 6=日曜日)
df['DayOfWeek'] = df['Date'].dt.dayofweek

# 月 (1=1月, 12=12月)
df['Month'] = df['Date'].dt.month

# 週番号 (1から52または53)
df['WeekOfYear'] = df['Date'].dt.isocalendar().week


# 6. テクニカル指標の計算
# RSI (Relative Strength Index) の計算
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# 14日間のRSIを計算して追加
df['RSI'] = calculate_rsi(df)

# ボリンジャーバンドの計算
def calculate_bollinger_bands(data, window=20):
    sma = data['Close'].rolling(window=window).mean()
    rstd = data['Close'].rolling(window=window).std()
    upper_band = sma + 2 * rstd
    lower_band = sma - 2 * rstd
    return upper_band, lower_band

# 20日間のボリンジャーバンドを計算して追加
df['Upper_Band'], df['Lower_Band'] = calculate_bollinger_bands(df)

# MACD (Moving Average Convergence Divergence) の計算
# 12日EMA (Exponential Moving Average) と 26日EMA の差分を計算
df['MACD'] = df['Close'].ewm(span=12, adjust=False).mean() - df['Close'].ewm(span=26, adjust=False).mean()

# 9日EMA のシグナルラインを計算して追加
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()


# 7. ファンダメンタル指標の追加
# 出来高変化率 (Volume Change Rate) の計算
df['Volume_Change'] = df['Volume'].pct_change()

# 出来高の移動平均 (5日, 10日) の計算
df['Volume_MA_5'] = df['Volume'].rolling(window=5).mean()
df['Volume_MA_10'] = df['Volume'].rolling(window=10).mean()


# 8. 株価の変動率の計算
# 日次変動率 (Daily Price Change) の計算
df['Daily_Change'] = df['Close'].pct_change()

# 高値と安値の変動率 (High-Low Range) の計算
df['High_Low_Range'] = (df['High'] - df['Low']) / df['Low']


# 9. 目的変数の設定
# 翌日の終値を目的変数として追加
df['Next_Close'] = df['Close'].shift(-1)

# 最後の行は目的変数がないため削除
df.dropna(inplace=True)

# データの先頭5行を表示して確認
print("\n特徴量を追加したデータの先頭5行:")
print(df.head())

In [ ]:
# 10. モデル構築の準備
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 説明変数（特徴量）の選択
X = df[['Open', 'High', 'Low', 'Close', 'Volume', 'MA_5', 'MA_10', 'MA_20', 'Volatility', 
        'DayOfWeek', 'Month', 'WeekOfYear', 'RSI', 'Upper_Band', 'Lower_Band', 'MACD', 
        'MACD_Signal', 'Volume_Change', 'Volume_MA_5', 'Volume_MA_10', 'Daily_Change', 
        'High_Low_Range']]

# 目的変数（ターゲット）の選択
y = df['Next_Close']

# 訓練データとテストデータに分割（80%訓練データ、20%テストデータ）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレスト回帰モデルの構築
model = RandomForestRegressor(n_estimators=100, random_state=42)

# モデルを訓練データで学習
model.fit(X_train, y_train)

# テストデータを用いて予測
y_pred = model.predict(X_test)

# 評価
# 平均二乗誤差（MSE）の計算
mse = mean_squared_error(y_test, y_pred)

# 平均絶対誤差（MAE）の計算
mae = mean_absolute_error(y_test, y_pred)

# 評価結果を表示
print("\nモデルの評価:")
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
